# Vllm

<center>
<img src="https://docs.vllm.ai/en/stable/assets/logos/vllm-logo-text-light.png" alt="vllm logo" width="400"/>
</center>

Vllm is less cool and has an ugly logo :) 

### 🏳️ Open the cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


launch a new terminal session

<center>
    <img src="https://icon-library.com/images/bash-icon/bash-icon-6.jpg" alt="bash is also cool" width="200"/>
</center>


and execute therein the following commands to load the previously prepared enviroment with vllm installed

```bash
# Set up Modules
source /etc/profile.d/modules.sh
module purge
module load Programming_Languages/python/3.11.4

# Activate conda environment
source /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/activate

# Enforce correct site-packages
unset PYTHONPATH
export PYTHONPATH=/pbs/throng/training/universite-hiver/wschooloptim/vllm/lib/python3.11/site-packages/

# Ollama and HF cache setting
export HUGGINGFACE_HUB_CACHE=/pbs/throng/training/universite-hiver/cache/huggingface
export OLLAMA_MODELS=/pbs/throng/training/universite-hiver/cache/ollama/models
```

launch an vllm server

```bash
vllm serve HuggingFaceH4/zephyr-7b-beta --max_model_len=10000 --guided_decoding_backend=xgrammar --seed=1  --tensor-parallel-size 1 &
```

Create one request to ollama's server:

```bash
curl -X POST http://localhost:8000/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
  "model": "HuggingFaceH4/zephyr-7b-beta",
  "messages": [{ "role": "user", "content": "Who won the last presidential elections in France?" }]
}'
```

Keep that terminal open for the rest of the tutorial, we will check therein ollama server logs.

### 🏴 Open another cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


and launch now the Notebook called `Python 3.13 (Ollama)`.

We will work from now in that notebook.

In [6]:
! xan tail hashtag_donaldtrump_6k_joebiden_25k_shuffled.csv | xan v 


Displaying 3/21 cols from 10 rows of <stdin>
┌───┬─────────────────────┬────────────────┬───┬───────────────────────────────┐
│ - │ created_at          │ tweet_id       │ … │ collected_at                  │
├───┼─────────────────────┼────────────────┼───┼───────────────────────────────┤
│ 0 │ 2020-10-16 01:19:38 │ 1.31691164952… │ … │ 2020-10-21 02:33:39.914774259 │
│ 1 │ 2020-10-15 18:06:56 │ 1.31680275680… │ … │ 2020-10-21 01:41:43.630185015 │
│ 2 │ 2020-10-15 00:07:37 │ 1.31653113467… │ … │ 2020-10-21 00:00:54.489613435 │
│ 3 │ 2020-10-15 08:38:01 │ 1.31665958287… │ … │ 2020-10-21 00:43:07.582933275 │
│ 4 │ 2020-10-16 01:30:04 │ 1.31691427389… │ … │ 2020-10-21 02:37:50.543179243 │
│ 5 │ 2020-10-15 08:03:42 │ 1.31665094463… │ … │ 2020-10-21 00:37:07.356253320 │
│ 6 │ 2020-10-16 10:29:56 │ 1.31705013636… │ … │ 2020-10-21 03:28:28.635848751 │
│ 7 │ 2020-10-16 07:39:55 │ 1.31700734863… │ … │ 2020-10-21 03:21:35.409676897 │
│ 8 │ 2020-10-15 16:39:48 │ 1.31678082824… │ … │ 2020-10-21 01:

##### Import python packages

In [3]:
import csv
import time
import asyncio
from openai import OpenAI
from string import Template
import nest_asyncio
nest_asyncio.apply()

##### Load inputs and create asynchronous iterator

In [7]:
input_file = "hashtag_donaldtrump_6k_joebiden_25k_shuffled.csv"
content_column = 'tweet'

start = time.time()
with open(input_file, 'r') as f:
    tweets = [d[content_column] for d in csv.DictReader(f)]
end = time.time()
print(f"Loading {len(tweets)} tweets tooks {end - start} seconds.")


Loading 31000 tweets tooks 0.20054149627685547 seconds.


##### Load choices and extra body dictionary with structured_outputs options

In [9]:
choices_file = "multiple_us.txt"
with open(choices_file, 'r') as f:
    choice = f.read().split("\n")

extra_body = {"structured_outputs": {"choice": choice}}

print(extra_body)

{'structured_outputs': {'choice': ['Biden', 'Trump', 'None', '']}}


##### Load instructions

In [10]:
instructions_file = "/pbs/throng/training/universite-hiver/wschooloptim/resinwinterschool/instructions.txt"
with open(instructions_file, 'r') as f:
    instructions = f.read()
print(instructions)

Please classify the following social media message (posted in the weeks leading up to the 2025 Chilean presidential election) according to whether it explicitly expresses the intention  of the author to vote for or calls for a vote for any of the candidates in that election: Jeannette Jara, José Antonio Kast, Johannes Kaiser, Evelyn Matthei, Franco Parisi, Eduardo Artés, Harold Mayne-Nichols, Marco Enríquez-Ominami (also known as MEO), or whether it expresses neither of these intentions. Your answer should be based solely on the information contained in the message. Do not confuse a simple mention of a candidate with an intention or call to vote for him. Do not assume that a message containing only positive opinions about a particular candidate explicitly expresses the intention to vote for that candidate. Do not assume that a message corresponding to the opinions or political positions of a particular candidate necessarily expresses the intention to vote for that candidate. Do not con

##### Create vllm openai client

In [11]:
client = OpenAI(
    base_url="http://localhost:8000/v1", # this is the same HOST given at ollama server launching
    api_key="EMPTY"  # required but unused
)

##### Create async functions to asynchronously request llm server


In [12]:
async def inputsIterator(start, end):
    for tweet in tweets[start:end]:
        yield tweet

In [13]:
async def doCompletetion(tweet):
    return client.chat.completions.create(
        model=client.models.list().data[0].id,
        messages=[{
            'role': 'user',
            'content': Template(instructions).substitute(tweet=tweet)
        }],
        extra_body=extra_body,
        max_completion_tokens=16,
        temperature=0.7,
        top_p=0.95,
    )

In [14]:
async def run_all(start, end):
    # Asynchronously call the function for each prompt
    tasks = [
        doCompletetion(tweet)
        async for tweet in inputsIterator(start, end)
    ]
    # Gather and run the tasks concurrently
    results = await asyncio.gather(*tasks)
    return results

In [15]:
# Run all courutines
init_idx = 3449
nb_tweets = 100

start = time.time()
results = asyncio.run(run_all(start=init_idx, end=init_idx + nb_tweets))
elapsed = time.time() - start

whole_db_nb_tweets = int(1e6)


print(f"""
Annotationg {nb_tweets} tweets took {elapsed} seconds,
this is {elapsed / nb_tweets} seconds per tweet or
{(1 / (24 * 3600)) * whole_db_nb_tweets * elapsed / nb_tweets} days for the whole database of {whole_db_nb_tweets} inputs.
""")


Annotationg 100 tweets took 6.010417222976685 seconds,
this is 0.06010417222976685 seconds per tweet or
0.6956501415482274 days for the whole database of 1000000 inputs.



> Annotationg 500 tweets took 27.607022047042847 seconds,
>
> tthis is 0.055214044094085694 seconds per tweet or
>
> 0.6670885399377953 days for the whole database of 1043873 inputs.